In [1]:
from mpi4py import MPI

In [2]:
import ipyparallel as ipp

cluster = ipp.Cluster()
cluster

<Cluster(cluster_id='1696462845-n4f1', profile='default')>

In [3]:
def show_clusters():
    clusters = ipp.ClusterManager().load_clusters() 
    print("{:15} {:^10} {}".format("cluster_id", "state", "cluster_file")) 
    for c in clusters:
        cd = clusters[c].to_dict()
        cluster_id = cd['cluster']['cluster_id']
        controller_state = cd['controller']['state']['state']
        cluster_file = getattr(clusters[c], '_trait_values')['cluster_file']
        print("{:15} {:^10} {}".format(cluster_id, controller_state, cluster_file))

In [4]:
show_clusters()

cluster_id        state    cluster_file
                 running   /Users/aaron/.ipython/profile_mpi/security/cluster-.json
1696460333-1igf  running   /Users/aaron/.ipython/profile_default/security/cluster-1696460333-1igf.json
1696461110-d0es  running   /Users/aaron/.ipython/profile_default/security/cluster-1696461110-d0es.json


In [5]:
from mpi4py import MPI
import math

# Inicializa MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()


In [11]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

slice_size = 1000000
total_slices = 50

# This is the master node.
if rank == 0:
    pi = 0
    slice = 0
    process = 1

    print (size)

    # Send the first batch of processes to the nodes.
    while process < size and slice < total_slices:
        comm.send(slice, dest=process, tag=1)
        print ("Sending slice",slice,"to process",process)
        slice += 1
        process += 1

    # Wait for the data to come back
    received_processes = 0
    while received_processes < total_slices:
        pi += comm.recv(source=MPI.ANY_SOURCE, tag=1)
        process = comm.recv(source=MPI.ANY_SOURCE, tag=2)
        print ("Recieved data from process", process)
        received_processes += 1

        if slice < total_slices:
            comm.send(slice, dest=process, tag=1)
            print ("Sending slice",slice,"to process",process)
            slice += 1

    # Send the shutdown signal
    for process in range(1,size):
        comm.send(-1, dest=process, tag=1)

    print ("Pi is ", 4.0 * pi)

# These are the slave nodes, where rank > 0. They do the real work
else:
    while True:
        start = comm.recv(source=0, tag=1)
        if start == -1: break

        i = 0
        slice_value = 0
        while i < slice_size:
            if i%2 == 0:
                slice_value += 1.0 / (2*(start*slice_size+i)+1)
            else:
                slice_value -= 1.0 / (2*(start*slice_size+i)+1)
            i += 1

        comm.send(slice_value, dest=0, tag=1)
        comm.send(rank, dest=0, tag=2)

1
